<a href="https://colab.research.google.com/github/jason147172/tibame/blob/master/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload() #upload kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

Saving kaggle.json to kaggle (1).json


In [ ]:
!kaggle competitions download -c titanic

test.csv: Skipping, found more recently modified local copy (use --force to force download)
gender_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
train.csv: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
# from urllib.request import urlretrieve
# url = "https://github.com/Elwing-Chou/ml0716/raw/master/train.csv"
# urlretrieve(url, "train.csv")

In [ ]:
import pandas as pd
test_df = pd.read_csv("test.csv", encoding="utf8")
train_df = pd.read_csv("train.csv", encoding="utf8")

In [ ]:
# pd.concat: 預設axis=0 垂直合併, axis=1 水平合併
data = pd.concat([train_df, test_df])
# .drop
data = data.drop(["PassengerId", "Survived"], axis=1)
data

In [ ]:
# Step1: 補缺失值 (補最可能的)
# data.isna().sum(): 統計所有類別含有缺失值的總和
s = data.isna().sum()
# 篩選 series[[True,False,....]]
s[s > 0]

Age          263
Fare           1
Cabin       1014
Embarked       2
dtype: int64

In [ ]:
# 類別:　補最常出現的類型　或　不補
# value_counts: 統計所有變數出現的次數 ; idxmax: 抓出出現次數最高的
most = data["Embarked"].value_counts().idxmax()
data["Embarked"] = data["Embarked"].fillna(most)

In [ ]:
#　數字: 補中位數
med = data.median().drop(["Pclass"]) # Pclass不是數字類型 
d = data.fillna(med)
d.isna().sum()

Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin       1014
Embarked       0
dtype: int64

In [ ]:
# Step2: One-Hot:類別要做, 但有兩種情況不用做
# (1.)類別只有兩種 ;(2.)類別已經具有大小關係
# (特殊) Pclass: 可以不用做One-Hot, 因為Data具有大小關係, 但補缺失值時須補最常出現的

# 先準備放類別的list
def name(n):
    n = n.split(",")[-1].split(".")[0]
    return n.strip()
v = data["Name"].apply(name).value_counts()
white = v.index[:4]

In [ ]:
def name(n):
    n = n.split(",")[-1].split(".")[0]
    n = n.strip()
    if n in white:
        return n
    else:
        return None
data["Name"] = data["Name"].apply(name)
data

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,Mr,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,Mrs,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,Miss,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,Mrs,female,35.0,1,0,113803,53.1000,C123,S
4,3,Mr,male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
413,3,Mr,male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1,None,female,39.0,0,0,PC 17758,108.9000,C105,C
415,3,Mr,male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,3,Mr,male,NaN,0,0,359309,8.0500,NaN,S


In [ ]:
# 新增判斷依據, 將相同Ticket的號碼合併再一起, 或許同個號碼和生存率有關
v = data["Ticket"].value_counts()
def ticket(t):
  global v
  return v[t]
data["Ticket"] = data["Ticket"].apply(ticket)

In [ ]:
# Cabin: 艙位
data = data.drop(["Cabin"], axis=1)

In [ ]:
# 如果想重製index: data.reset_index(drop=True)
data

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,3,Mr,male,22.0,1,0,1,7.2500,S
1,1,Mrs,female,38.0,1,0,2,71.2833,C
2,3,Miss,female,26.0,0,0,1,7.9250,S
3,1,Mrs,female,35.0,1,0,2,53.1000,S
4,3,Mr,male,35.0,0,0,1,8.0500,S
...,...,...,...,...,...,...,...,...,...
413,3,Mr,male,NaN,0,0,1,8.0500,S
414,1,None,female,39.0,0,0,3,108.9000,C
415,3,Mr,male,38.5,0,0,1,7.2500,S
416,3,Mr,male,NaN,0,0,1,8.0500,S


In [ ]:
data = pd.get_dummies(data)
data

,Pclass,Age,SibSp,Parch,Ticket,Fare,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.0,1,0,1,7.2500,0,0,1,0,0,1,0,0,1
1,1,38.0,1,0,2,71.2833,0,0,0,1,1,0,1,0,0
2,3,26.0,0,0,1,7.9250,0,1,0,0,1,0,0,0,1
3,1,35.0,1,0,2,53.1000,0,0,0,1,1,0,0,0,1
4,3,35.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,3,NaN,0,0,1,8.0500,0,0,1,0,0,1,0,0,1
414,1,39.0,0,0,3,108.9000,0,0,0,0,1,0,1,0,0
415,3,38.5,0,0,1,7.2500,0,0,1,0,0,1,0,0,1
416,3,NaN,0,0,1,8.0500,0,0,1,0,0,1,0,0,1


In [ ]:
# 取列: iloc(看實際是第幾列), loc(看列標籤)
# [第一列, 第二列, ...]
x_train = data.iloc[:len(train_df)]
x_predict = data.iloc[:len(train_df)]
y_train = train_df["Surviced"]

In [ ]:
from sklearn.preprocessing